# Introduction to Bedrock - Text Generation Sample : Creating Product Descriptions

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a text question and answer (Q&A) example. We show how to use Bedrock's Foundational Models to answer questions after reading a passage

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Question and Answer](#2.-Reading-Comprehension)

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [1]:
bedrock_region = "us-west-2" 

In [2]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [3]:
import boto3
import json
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [4]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '2399dcc0-e7ae-4ab3-9ab9-317e3e9ba11c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Jul 2023 04:58:08 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2399dcc0-e7ae-4ab3-9ab9-317e3e9ba11c'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Text Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Reading Comprehension

#### Let's now try out the Amazon Bedrock model to have it create answer a question after reading a passage

In [5]:
prompt_data ="""Read the following passage and answer the questions that follow: 

            The Supreme Court order banning the registration of diesel vehicles beyond 2000 cc in and around New Delhi points to the dire situation and need for drastic steps. 
            It is not necessary, therefore, to discuss the pros and cons of a three-month long ban, on a trial basis. The court consciously targeted only luxury vehicles, sports 
            utility vehicles (SUVs), besides big trucks, affordable to affluent people, sparing the common man. It concurred with the argument that diesel vehicles pollute the most, 
            contributing to pollution. Experts have maintained that "diesel exhaust consists of 10-100 times (that is, one to two orders of magnitude) more particles than petrol". 
            Moreover, diesel exhaust is said to contain smaller particles which are more hazardous as they enter the deeper recesses of lungs and stay longer causing serious harm to health. 
            The industry, both automobile manufacturers and distributors, predictably raised a hue and cry claiming the decision would affect trade and commerce and movement and supply 
            of essential commodities in the city. The industry had been trying to pass the buck by lobbying on two grounds - one, that diesel is not as dirty as environmentalists claim it to 
            be and, two, there were other more dangerous sources of pollution which should be tackled first. When faced with an adverse judgment, Corporates seek new scientific studies and proof. 

            1. According to the passage, the Supreme Court has exempted which type of diesel vehicles from the registration banning list? 
            a) Luxury vehicles 
            b) Sports utility vehicles (SUVs) 
            c) Vehicles affordable only by affluent people 
            d) Vehicles with a capacity of below 2000 cc 
            Correct answer:"""#If you'd like to try your own prompt, edit this parameter!

### Let's start by using the Amazon Titan Large Model

In [8]:
body = json.dumps(
    {
        "inputText": prompt_data,
        "textGenerationConfig":
        {
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        }
    }
)
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))

 d) Vehicles with a capacity of below 2000 cc.


### Let's try the prompt with the Anthropic Claude Instant Model on Bedrock

In [5]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 50})
modelId = 'anthropic.claude-instant-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 d) Vehicles with a capacity of below 2000 cc

Analysis: The passage states "It consciously targeted only luxury vehicles, sports utility vehicles (SUVs), besides big trucks, affordable to affluent people, sparing the common man


### Let's try the prompt with the Jurrasic Grande Model on Bedrock

In [6]:
body = json.dumps({"prompt": prompt_data})
modelId = 'ai21.j2-grande-instruct' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completions')[0].get('data').get('text'))

 c) Vehicles affordable only by affluent people

Answer (c) is correct as the
